In [ ]:
from utils import VDCNN
from utils import make_dataloader, run_model

In [ ]:
train_fname = '../../data/yelp_review_polarity_csv/train.csv'
test_fname = '../../data/yelp_review_polarity_csv/test.csv'
num_epochs = 15
depth = 9
num_class = 2

dataloaders = make_dataloader(train_fname, test_fname)

model = VDCNN(depth=depth, num_class=num_class)

run_model(model, dataloaders, num_epochs)